In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

data
split



The bson files for this competition contain a list of dictionaries, one dictionary per product. Each dictionary contains a product id (key: _id), the category id of the product (key: category_id), and between 1-4 images, stored in a list (key: imgs). Each image list contains a single dictionary per image, which uses the format: {'picture': b'...binary string...'}. 

The bson file can be read and processed iteratively. The following code shows how to read the data from the `train_example.bson` file.

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
# Simple data processing

data = bson.decode_file_iter(open('../input/data/train.bson', 'rb'))

prod_to_category = dict()

for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = { 'category_id': category_id, 'num_imgs': len(d['imgs']) }
    #for e, pic in enumerate(d['imgs']):
    #    picture = imread(io.BytesIO(pic['picture']))
        # do something with the picture, etc

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id', 1: 'num_imgs'}, inplace=True)

In [4]:
prod_to_category.head()

,category_id,num_imgs
_id,,
0,1000010653,1
1,1000010653,1
2,1000004079,1
3,1000004141,1
4,1000015539,1


In [5]:
prod_to_category.to_csv('../input/split/train_images.csv')

In [3]:
import cv2
import pickle
data = bson.decode_file_iter(open('../input/data/test.bson', 'rb'))

"""
num_dict = {}
idx=0
for c, d in enumerate(data):
    product_id = d['_id']
    #category_id = d['category_id'] # This won't be in Test data
     
    for e, pic in enumerate(d['imgs']):
        #picture = cv2.imread(pic['picture'])
        num_dict[idx] = product_id # 'category_id': category_id,
        idx+=1
    #idx+=1
#num_df = pd.DataFrame.from_dict(num_dict, orient='index').sample(frac=1)
#num_df.index.name = 'num'
#num_df.rename(columns={0: 'num_imgs', 1: '_id'}, inplace=True) #0: 'category_id',
pickle.dump(num_dict, open('./index_to_product.pkl', 'wb'))
num_dict
"""
prod_to_category = dict()

for c, d in enumerate(data):
    product_id = d['_id']
    #category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = { 'num_imgs': len(d['imgs']) }
prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'num_imgs'}, inplace=True)

In [4]:
prod_to_category.head()

,num_imgs
_id,
10,3
14,1
21,1
24,1
27,1


In [5]:
prod_to_category.to_csv('../input/split/test_images.csv')

In [6]:
BASE_DIR = '/home/cory/Kaggle/Cdiscount/'
OUT_DIR = BASE_DIR+'output/resnet50-pretrain/'
out_dir = OUT_DIR
test_dir = out_dir +'/submissions/'
predictions_arr = np.load(test_dir+'/results.csv.npy')
train_df = pd.read_csv(BASE_DIR+'input/split/train_images_split.csv')
valid_df = pd.read_csv(BASE_DIR+'input/split/valid_images_split.csv')
classes_df = train_df['category_id'].append(valid_df['category_id'])
classes_df = classes_df.drop_duplicates()
classes_dict = classes_df.to_dict()
classes_dict = { i : classes_dict[k] for i, k in enumerate(classes_dict) }
predictions_df = pd.DataFrame(predictions_arr, columns=["label"])
predictions_df["label"] = predictions_df["label"].apply(lambda x: classes_dict[x])
predictions_df.head()

,label
0,1000014298
1,1000004307
2,1000012342
3,1000001585
4,1000005732


In [12]:
prod_dict = {}
for i in num_dict:
    prod = num_dict[i]
    if prod in prod_dict:
        #print(prod_dict)
        prod_dict[prod].append(i)
    else:
        prod_dict[prod] = [i]
prod_dict

{10: [0, 1, 2],
 14: [3],
 21: [4],
 24: [5],
 27: [6],
 29: [7],
 32: [8, 9, 10, 11],
 49: [12],
 67: [13],
 85: [14],
 86: [15],
 93: [16],
 94: [17],
 96: [18, 19, 20],
 110: [21, 22, 23, 24],
 113: [25],
 115: [26, 27, 28, 29],
 117: [30],
 135: [31, 32],
 136: [33],
 141: [34],
 143: [35],
 145: [36, 37],
 147: [38],
 165: [39],
 169: [40, 41],
 180: [42],
 190: [43],
 196: [44],
 202: [45],
 205: [46],
 207: [47],
 210: [48],
 213: [49],
 216: [50],
 218: [51],
 231: [52],
 236: [53],
 247: [54],
 248: [55],
 254: [56],
 256: [57, 58, 59, 60],
 259: [61, 62, 63, 64],
 261: [65],
 272: [66],
 276: [67],
 278: [68],
 279: [69],
 285: [70, 71, 72, 73],
 286: [74],
 290: [75],
 297: [76, 77],
 298: [78],
 303: [79, 80, 81, 82],
 304: [83],
 325: [84],
 326: [85],
 328: [86],
 332: [87],
 335: [88],
 338: [89],
 339: [90],
 346: [91, 92],
 349: [93, 94],
 353: [95],
 356: [96, 97, 98, 99],
 360: [100],
 362: [101],
 365: [102, 103, 104, 105],
 367: [106, 107, 108, 109],
 368: [110, 11

In [15]:
prod_cat_dict = {}
for i in prod_dict:
    prod_cat_dict[i] = []
    for index in prod_dict[i]:
        prod_cat_dict[i].append(predictions_df.iloc[index]["label"])
prod_cat_dict

{10: [1000014298, 1000004307, 1000012342],
 14: [1000001585],
 21: [1000005732],
 24: [1000016738],
 27: [1000017911],
 29: [1000010918],
 32: [1000008933, 1000009663, 1000001838, 1000005878],
 49: [1000012145],
 67: [1000011717],
 85: [1000003787],
 86: [1000004055],
 93: [1000002890],
 94: [1000011401],
 96: [1000005712, 1000005744, 1000011427],
 110: [1000001266, 1000012965, 1000015736, 1000014222],
 113: [1000015286],
 115: [1000003945, 1000001831, 1000014322, 1000003983],
 117: [1000004371],
 135: [1000008933, 1000013768],
 136: [1000014252],
 141: [1000014030],
 143: [1000003945],
 145: [1000015704, 1000004019],
 147: [1000014810],
 165: [1000010608],
 169: [1000005842, 1000009054],
 180: [1000004147],
 190: [1000008778],
 196: [1000005986],
 202: [1000011427],
 205: [1000000568],
 207: [1000003949],
 210: [1000010629],
 213: [1000001780],
 216: [1000014156],
 218: [1000006062],
 231: [1000012989],
 236: [1000016738],
 247: [1000010401],
 248: [1000000457],
 254: [1000011566],
 2

In [4]:
num_df.head()

,_id,ind
num,,
1819927,13169075,0
2564718,19156311,3
2799664,21125310,0
3068668,23405970,0
223806,1329738,2


In [5]:
num_df.to_csv('../input/split/test_images.csv')

In [10]:
# Simple data processing

data = bson.decode_file_iter(open('../input/data/train.bson', 'rb'))

for i, d in enumerate(data):
    print(d)
    if i == 2:
        break

{'category_id': 1000010653, 'imgs': [{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00\xb4\x00\xb4\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9

In [6]:
plt.imshow(picture);

For more efficient use of your resources, you can use the `multiprocessing` module to read and process the bson file.

Inspiration for this code is from:  https://stackoverflow.com/questions/43078980/python-multiprocessing-with-generator Note this may be slower on a small file, due to the overhead setting up the workers, but will be significantly faster for the large files.

In [11]:
NCORE =  8

prod_to_category = mp.Manager().dict() # note the difference

def process(q, iolock):
    while True:
        d = q.get()
        if d is None:
            break
        product_id = d['_id']
        category_id = d['category_id']
        prod_to_category[product_id] = category_id
        for e, pic in enumerate(d['imgs']):
            picture = imread(io.BytesIO(pic['picture']))
            # do something with the picture, etc
    
q = mp.Queue(maxsize=NCORE)
iolock = mp.Lock()
pool = mp.Pool(NCORE, initializer=process, initargs=(q, iolock))

# process the file

data = bson.decode_file_iter(open('../input/data/train.bson', 'rb'))
for c, d in enumerate(data):
    q.put(d)  # blocks until q below its max size

# tell workers we're done

for _ in range(NCORE):  
    q.put(None)
pool.close()
pool.join()

# convert back to normal dictionary
prod_to_category = dict(prod_to_category)

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Process ForkPoolWorker-25:
Process ForkPoolWorker-22:
Process ForkPoolWorker-24:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "<ipython-input-11-449ff600a4ea>", line 14, in process
    picture = imread(io.BytesIO(pic['picture']))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py", line 61, in imread
    img = call_plugin('imread', fname, plugin=plugin, **pl

KeyboardInterrupt: 

  File "/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py", line 61, in imread
    img = call_plugin('imread', fname, plugin=plugin, **plugin_args)
  File "/usr/local/lib/python3.5/dist-packages/skimage/io/_plugins/pil_plugin.py", line 40, in imread
    return pil_to_ndarray(im, dtype=dtype, img_num=img_num)
  File "/usr/local/lib/python3.5/dist-packages/skimage/io/_plugins/pil_plugin.py", line 53, in pil_to_ndarray
    im.getdata()[0]
KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/skimage/io/manage_plugins.py", line 211, in call_plugin
    return func(*args, **kwargs)
  File "<ipython-input-11-449ff600a4ea>", line 12, in process
    prod_to_category[product_id] = category_id
  File "/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py", line 61, in imread
    img = call_plugin('imread', fname, plugin=plugin, **plugin_args)
  File "/home/cory/.local/lib/python3.5/site-packages/PIL/Image.py", line 1153, in getdata
    self.load()
  File "/usr/local/lib/pytho

In [ ]:
prod_to_category.head()

In [10]:
prod_to_category.to_csv('../input/split/train_images.csv')